<a href="https://colab.research.google.com/github/HoseinNekouei/sentiment_analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import torch
import pandas as pd
from google.colab import userdata
import matplotlib.pyplot as plt
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
from transformers import AutoTokenizer, AutoModelForSequenceClassification

userdata.get('HF_TOKEN')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class SentimentAnalysisModel:
    def __init__(self, checkpoint, epochs=2, batch_size=32, n_splits=5):
        self.checkpoint = checkpoint
        self.epochs = epochs
        self.batch_size = batch_size
        self.n_splits = n_splits
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer, self.model = self.load_model_and_tokenizer()
        self.optimizer = AdamW(self.model.parameters(), lr=5e-5)

    def load_model_and_tokenizer(self):
        """Load the tokenizer and model from the specified checkpoint."""
        tokenizer = AutoTokenizer.from_pretrained(self.checkpoint)
        model = AutoModelForSequenceClassification.from_pretrained(self.checkpoint, device_map='auto')
        model.to(self.device)
        return tokenizer, model

    def tokenize_texts(self, texts):
        """Tokenize a list of texts using the provided tokenizer."""
        if not isinstance(texts, list):
            texts = texts.tolist()

        encodings = self.tokenizer(
            texts,
            padding=True,
            max_length=256,
            truncation=True,
            return_tensors='pt'
        )
        return encodings

    def prepare_dataloaders(self, input_ids, attention_mask, labels, train_idx, test_idx):
        """Prepare training and test dataloaders."""
        train_dataset = TensorDataset(input_ids[train_idx], attention_mask[train_idx], labels[train_idx])
        test_dataset = TensorDataset(input_ids[test_idx], attention_mask[test_idx], labels[test_idx])

        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size)
        return train_loader, test_loader

    def train_model(self, train_loader):
        """Train the model for the specified number of epochs."""
        self.model.train()
        for epoch in range(self.epochs):
            for i, batch in enumerate(train_loader):
                self.optimizer.zero_grad()
                input_ids_batch, attention_mask_batch, labels_batch = [b.to(self.device) for b in batch]
                outputs = self.model(input_ids_batch, attention_mask=attention_mask_batch, labels=labels_batch)
                loss = outputs.loss
                loss.backward()
                self.optimizer.step()

                if (i + 1) % 100 == 0:
                    denominator = math.ceil(len(train_loader.dataset) / self.batch_size)
                    print(f'[Epoch: {epoch + 1}] -> Batch: [{i + 1}/{denominator}]')

            print(f'[Epoch: {epoch + 1}] -> Batch: [{denominator}/{denominator}]')

    def evaluate_model(self, test_loader):
        """Evaluate the model on the validation set and return accuracy."""
        self.model.eval()
        correct = 0
        total = 0
        all_preds= []
        all_labels= []

        with torch.no_grad():
            for batch in test_loader:
                input_ids_batch, attention_mask_batch, labels_batch = [b.to(self.device) for b in batch]

                outputs = self.model(input_ids_batch, attention_mask=attention_mask_batch)
                preds = torch.argmax(outputs.logits, dim=1)
                probs= torch.softmax(outputs.logits, dim=1)

                all_preds.append(probs)
                all_labels.append(labels_batch)

                correct += torch.sum(preds == labels_batch).item()
                total += len(labels_batch)

        # Concatenate all predictions and labels
        all_preds = torch.cat(all_preds, dim=0)
        all_labels = torch.cat(all_labels, dim=0)

        accuracy = correct / total

        # Calculate F1-score
        f1 = self.calculate_f1_score(all_preds, all_labels)

        return accuracy, f1, all_preds, all_labels

    def calculate_f1_score(self, preds, labels):
        """Calculate the F1-score for multi-class classification."""
        preds = torch.argmax(preds, dim=1)
        tp = (preds == labels).sum().item()
        fp = (preds != labels).sum().item()
        fn = (labels != preds).sum().item()

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        return f1

    def plot_roc_curve(self, y_true, y_scores, n_classes):
        """Plot the ROC curve for multi-class classification."""
        fpr = dict()
        tpr = dict()
        roc_auc = dict()

        # Compute ROC curve and ROC area for each class
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true.cpu().numpy() == i, y_scores[:, i].cpu().numpy())
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Plot ROC curve
        plt.figure()
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

        plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve')
        plt.legend(loc="lower right")
        plt.show()

    def kfold_cross_validation(self, train_data):
        """Perform k-fold cross-validation on the provided dataset."""
        # Tokenize texts and prepare labels
        texts = train_data['text']
        labels = torch.tensor(train_data['labels'].tolist()).to(self.device)
        encodings = self.tokenize_texts(texts)
        input_ids = encodings['input_ids'].to(self.device)
        attention_mask = encodings['attention_mask'].to(self.device)

        # Initialize KFold
        kfold = KFold(n_splits=self.n_splits, shuffle=True, random_state=42)
        accuracy_scores = []
        f1_scores = []
        all_preds = []
        all_labels = []

        # Perform k-fold cross-validation
        for fold, (train_idx, test_idx) in enumerate(kfold.split(input_ids)):
            print(f"Fold {fold + 1}...")

            # Prepare dataloaders
            train_loader, test_loader = self.prepare_dataloaders(input_ids, attention_mask, labels, train_idx, test_idx)

            # Train the model
            self.train_model(train_loader)

            # Evaluate the model
            fold_accuracy, fold_f1, fold_preds, fold_labels = self.evaluate_model(test_loader)
            accuracy_scores.append(fold_accuracy)
            f1_scores.append(fold_f1)
            all_preds.append(fold_preds)
            all_labels.append(fold_labels)

            print(f"Validation Accuracy: {fold_accuracy:.4f}")
            print(f"Validation F1-Score: {fold_f1:.4f}")
            print('=' * 30)

        # Calculate and print final results
        accuracy_scores = torch.tensor(accuracy_scores)
        f1_scores = torch.tensor(f1_scores)
        print(f"Cross-validation accuracy scores: {accuracy_scores}")
        print(f"Mean accuracy: {torch.mean(accuracy_scores):.4f}")
        print(f"Standard deviation (accuracy): {torch.std(accuracy_scores):.4f}")
        print(f"Cross-validation F1-scores: {f1_scores}")
        print(f"Mean F1-score: {torch.mean(f1_scores):.4f}")
        print(f"Standard deviation (F1-score): {torch.std(f1_scores):.4f}")

        # Concatenate predictions and labels from all folds
        all_preds = torch.cat(all_preds, dim=0)
        all_labels = torch.cat(all_labels, dim=0)

        # Plot ROC curve
        n_classes = len(torch.unique(all_labels))
        self.plot_roc_curve(all_labels, all_preds, n_classes)

        # Save the model
        model_save_path = '/content/sentiment_analysis/sentiment_model_batch32_K5.pt'
        torch.save(self.model, model_save_path)
        print(f"Model saved to {model_save_path}!")

In [ ]:
if __name__ == "__main__":
    # Clone the repository and load the data
    !git clone 'https://github.com/HoseinNekouei/sentiment_analysis.git'
    file_path = '/content/sentiment_analysis/twitter_train_corpus.csv'

    # Constants
    nrows= None
    N_SPLITS= 5 # k-fold
    EPOCHS = 2
    BATCH_SIZE = 32

    train_data = pd.read_csv(file_path, nrows= nrows) # Load your training data
    print(train_data.info())

    # Initialize the SentimentAnalysisModel class
    checkpoint= 'cardiffnlp/twitter-roberta-base-sentiment-latest'
    model = SentimentAnalysisModel(checkpoint, batch_size= BATCH_SIZE, epochs= EPOCHS, n_splits= N_SPLITS)

    # Perform k-fold cross-validation
    model.kfold_cross_validation(train_data)

fatal: destination path 'sentiment_analysis' already exists and is not an empty directory.


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fold 1...
[Epoch: 1] -> Batch: [1/5]
[Epoch: 1] -> Batch: [2/5]
[Epoch: 1] -> Batch: [3/5]
